### 1. 读取数据

In [6]:
import polars as pl
from typing import Dict, Tuple, List, Set
from collections import defaultdict
import heapq
from pathlib import Path
import math
import pickle
from numba import jit
from tqdm import tqdm
import numpy as np

# 数据存储路径和保存路径
offline_path = "/data1/zxh/news_rec/offline_data"
online_path = "/data1/zxh/news_rec/online_data"
save_path = '/data1/zxh/news_rec/temp_results'

In [7]:
def get_all_expose_df(offline_path="/data1/zxh/news_rec/offline_data", 
                     online_path="/data1/zxh/news_rec/online_data",
                     offline=True):
    """
    获取曝光数据集
    
    参数：
    - data_path: 训练数据路径
    - eva_path: 评估数据路径
    - offline: 是否为离线训练模式，若为 False，则合并训练集和验证集
    
    返回：
    - train_df: 训练数据集
    - test_df: 测试数据集
    """
    # 读取训练和验证数据
    train_df = pl.read_ipc(f"{offline_path}/train_data_offline.ipc")
    val_df = pl.read_ipc(f"{offline_path}/val_data_offline.ipc")
    test_df = pl.read_ipc(f"{online_path}/test_data_online.ipc")

    # 处理 offline/online 逻辑
    if offline:
        return train_df, val_df, test_df
    else:
        # 线上模式：合并 train 和 val
        train_df = pl.concat([train_df, val_df], how="vertical")
        return train_df, test_df

In [8]:
offline = True
if offline:
    train_df, val_df, test_df = get_all_expose_df(offline_path, online_path,offline)
else:
    train_df, test_df = get_all_expose_df(offline_path, online_path,offline)

### 2. 多路召回通道

#### 2.1 ItemCF召回通道

In [9]:
def get_user_item_time(expose_df: pl.DataFrame) -> Dict[int, List[Tuple[int, int]]]:
    """
    构建用户点击行为时间线字典

    根据用户点击日志生成结构字典，键为用户ID，值为按时间排序的(文章ID, 时间戳)元组列表
    
    参数:
        expose_df (pl.DataFrame): 必须包含以下列:
            - user_id: 整数类型
            - article_id: 整数类型  
            - expose_time: float（时间戳, ms）
    
    返回:
        Dict[int, List[Tuple[int, int]]]: 用户点击时间线字典
            示例: {123: [(456, 0.162), (789, 0.179)]}
    """
    # 筛选出点击序列，并按时间戳排序，确保点击行为的时序性
    sorted_df = expose_df.filter(pl.col("is_clicked") == 1).sort("expose_time")

    # 计算 expose_time 最小值和最大值，方便后续进行归一化
    min_value, max_value = sorted_df["expose_time"].min(), sorted_df["expose_time"].max()

    # 对 sorted_df 的 expose_time 进行 Min-Max 归一化
    sorted_df = sorted_df.with_columns(
    ((pl.col("expose_time") - min_value) / (max_value - min_value)).alias("expose_time"))

    # 按 user_id 分组，并将 (article_id, expose_time) 组合成列表
    user_item_time_dict = (
        sorted_df
        .group_by("user_id")
        .agg(pl.struct("article_id", "expose_time").alias("click_list"))
        .to_dict(as_series=False)  # 转换为 Python 字典
    )

    # 解析 Polars 结构化数据，转换为标准 Python 格式
    return {
        user_id: [(entry["article_id"], entry["expose_time"]) for entry in click_list]
        for user_id, click_list in zip(user_item_time_dict["user_id"], user_item_time_dict["click_list"])
    }

In [6]:
def itemcf_sim(user_item_time_dict: Dict[int, List[Tuple[int, int]]], save_path: str, offline : bool = True) -> Dict[int, List[Tuple[int, float]]]:
    """
    计算文章与文章之间的相似性矩阵（基于物品的协同过滤 ItemCF）。

    :param user_item_time_dict: Dict[int, List[tuple[int, int]]]，用户点击的文章序列 {user_id: [(article_id, expose_time), ...]}
               - user_id (int): 用户 ID
               - article_id (int): 文章 ID
               - expose_time (float): 点击时间,这里已经筛选了点击序列
    :param save_path: str，相似性矩阵和倒排索引存储路径
    :param offline: bool, 是否为离线模式
    :return: Dict[int, List[Tuple[int, float]]]，每个物品的相似物品倒排索引表
             格式：
             {
                item1: [(item2, 相似度值), (item3, 相似度值), ...],
                item2: [(item1, 相似度值), (item3, 相似度值), ...],
                ...
             }
    """
    # 文章相似度字典，存储文章共现关系
    i2i_sim: Dict[int, Dict[int, float]] = defaultdict(dict)
    item_cnt: Dict[int, int] = defaultdict(int)  # 记录每篇文章的点击次数

    # 遍历每个用户的点击序列，构建文章共现关系
    for user, item_time_list in tqdm(user_item_time_dict.items(), desc="Computing item co-occurrence"):
        item_count = len(item_time_list) # 当前用户点击的文章总数

        active_weight = 1 / math.log1p(item_count + 1)  # 计算 log(1 + |I_u|) 作为用户活跃度的权重

        for loc_1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1  # 统计每篇文章的点击次数，用于计算文章流行度

            for loc_2, (j, j_click_time) in enumerate(item_time_list):
                if i == j:
                    continue  # 跳过自身，避免自相似度计算

                # 计算文章 i 和 j 之间的位置信息影响 (方向性影响 c·β^(|l_i - l_j| - 1))
                loc_alpha = 1.0 if loc_2 > loc_1 else 0.7  # 正向浏览时 c=1.0，反向浏览时 c=0.7
                loc_weight = loc_alpha * (0.8 ** (np.abs(loc_2 - loc_1) - 1))

                # 计算文章 i 和 j 之间的时间间隔影响 (exp(-α * |t_i - t_j|))
                time_weight = np.exp(-15000 * np.abs(i_click_time - j_click_time)) 
                
                i2i_sim[i].setdefault(j, 0)
                i2i_sim[i][j] += loc_weight * time_weight * active_weight  # 计算共现权重

    # 归一化相似性矩阵
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])


    # 构建倒排索引：每个物品的相似物品
    inverted_index: Dict[int, List[Tuple[float, int]]] = defaultdict(list)
    for i, related_items in i2i_sim.items():
        # 使用堆维护相似物品的倒排索引pair
        topk_items = heapq.nlargest(len(related_items), related_items.items(), key=lambda x: x[1])
        inverted_index[i] = [(j, sim) for j, sim in topk_items]

    # 存储相似性矩阵和倒排索引
    mode = "offline" if offline else "online"

    # 存储相似性矩阵和倒排索引
    with open(f"{save_path}/itemcf_i2i_sim_{mode}.pkl", "wb") as f:
        pickle.dump(i2i_sim, f)

    with open(f"{save_path}/itemcf_inverted_index_{mode}.pkl", "wb") as f:
        pickle.dump(inverted_index, f)

    return inverted_index

In [12]:
# 调用ItemCF召回通道进行新闻召回
user_item_time_dict = get_user_item_time(train_df) # 用户点击行为时间线字典

In [ ]:
inverted_index = itemcf_sim(user_item_time_dict, save_path, offline) # 构建物品相似物品的倒排索引

Computing item co-occurrence:   0%|          | 0/869946 [00:00<?, ?it/s]

Computing item co-occurrence: 100%|██████████| 869946/869946 [3:25:21<00:00, 70.60it/s]   


#### 2.2 UserCF召回通道

In [3]:
def get_item_user(expose_df: pl.DataFrame) -> Dict[int, List[int]]:
    """
    构建物品的用户点击字典

    根据用户点击日志生成结构字典，键为文章ID，值为点击该文章的用户ID列表。

    参数:
        expose_df (pl.DataFrame): 必须包含以下列:
            - user_id: 整数类型
            - article_id: 整数类型
            - is_clicked: 是否点击 (1 表示点击)

    返回:
        Dict[int, List[int]]: 物品的用户点击字典
            示例: {456: [123, 789]}
    """
    # 筛选点击记录
    filtered_df = expose_df.filter(pl.col("is_clicked") == 1)

    # 按 article_id 分组，获取对应的用户列表
    item_user_dict = (
        filtered_df
        .group_by("article_id")
        .agg(pl.col("user_id").alias("user_list"))
        .to_dict(as_series=False)  # 转换为 Python 字典
    )

    # 解析 Polars 结果，转换为标准 Python 格式
    return {
        article_id: user_list
        for article_id, user_list in zip(item_user_dict["article_id"], item_user_dict["user_list"])
    }

In [4]:
def usercf_sim(item_user_dict: Dict[int, List[Tuple[int, int]]], save_path: str, offline: bool = True) -> Dict[int, List[Tuple[int, float]]]:
    """
    计算用户之间的相似性矩阵（基于用户的协同过滤 UserCF），考虑用户活跃度和物品流行度。

    :param item_user_dict: Dict[int, List[int]]，物品的用户点击序列 {item_id: [user_id, ...]}
    :param save_path: str，相似性矩阵和倒排索引存储路径
    :param offline: bool, 是否为离线模式
    :return: Dict[int, List[Tuple[int, float]]]，每个用户的相似用户倒排索引表
             {
                user1: [(user2, 相似度值), (user3, 相似度值), ...],
                user2: [(user1, 相似度值), (user3, 相似度值), ...],
                ...
             }
    """
    u2u_sim: Dict[int, Dict[int, float]] = defaultdict(dict)  # 用户相似度矩阵
    user_item_cnt: Dict[int, int] = defaultdict(int)  # 记录每个用户的点击物品数量
        
    # 遍历每个物品的用户集合，构建用户共现矩阵
    for item, users in tqdm(item_user_dict.items(), desc="Building user co-occurrence matrix"):
        log_weight = 1.0 / math.log1p(len(users) + 1) # log(1 + |U_i|) 代表物品的流行度
        for u in users:
            user_item_cnt[u] += 1 # 统计每个用户点击的文章数量，用于计算用户的活跃度
            for v in users:
                if u == v:
                    continue
                
                u2u_sim[u].setdefault(v, 0)
                u2u_sim[u][v] += log_weight  # 计算相似度分子部分

    # 归一化相似度矩阵
    for u, related_users in tqdm(u2u_sim.items(), desc="Normalizing similarity matrix"):
        for v, sim_uv in related_users.items():
            u2u_sim[u][v] = sim_uv / math.sqrt(user_item_cnt[u] * user_item_cnt[v])

    # 构建倒排索引：每个用户的相似用户
    inverted_index: Dict[int, List[Tuple[int, float]]] = defaultdict(list)
    for u, related_users in tqdm(u2u_sim.items(), desc="Building user inverted index"):
        topk_users = heapq.nlargest(len(related_users), related_users.items(), key=lambda x: x[1])
        inverted_index[u] = [(v, sim) for v, sim in topk_users]

    # 存储相似性矩阵和倒排索引
    mode = "offline" if offline else "online"
    with open(f"{save_path}/usercf_u2u_sim_{mode}.pkl", "wb") as f:
        pickle.dump(u2u_sim, f)
    with open(f"{save_path}/usercf_inverted_index_{mode}.pkl", "wb") as f:
        pickle.dump(inverted_index, f)

    return inverted_index

In [9]:
# 调用UserCF召回通道
item_user_dict = get_item_user(train_df) # 构建物品的用户点击字典

In [ ]:
inverted_index = usercf_sim(item_user_dict, save_path, offline) # 计算用户之间的相似性矩阵，返回用户的倒排索引

#### 2.3 Swing召回通道

In [ ]:
def swing_sim(user_item_time_dict: Dict[int, List[Tuple[int, int]]], save_path: str, offline: bool = True) -> Dict[int, List[Tuple[int, float]]]:
    """
    计算基于 Swing 召回的物品相似性矩阵。

    :param user_item_time_dict: Dict[int, List[tuple[int, int]]]，用户点击的文章序列 {user_id: [(article_id, expose_time), ...]}
               - user_id (int): 用户 ID
               - article_id (int): 文章 ID
               - expose_time (float): 点击时间, 这里已经筛选了点击序列
    :param save_path: str，相似性矩阵和倒排索引存储路径
    :param offline: bool, 是否为离线模式
    :return: Dict[int, List[Tuple[int, float]]]，每个物品的相似物品倒排索引表
             {
                item1: [(item2, 相似度值), (item3, 相似度值), ...],
                item2: [(item1, 相似度值), (item3, 相似度值), ...],
                ...
             }
    """

    # 1. 记录每个物品被哪些用户点击过，以及每个用户的点击记录
    item_user_cnt: Dict[int, int] = defaultdict(int)  # 记录每个物品的点击次数，即为 |U_i|
    user_co_items: Dict[Tuple[int, int], List[int]] = defaultdict(list)  # 记录用户组与共现物品的字典

    # 遍历用户点击序列，构建物品的共现关系
    for user, item_time_list in tqdm(user_item_time_dict.items(), desc="Building item-user interaction matrix"):
        for item, timestamp in item_time_list:
            item_user_cnt[item] += 1  # 统计物品点击次数

        # 计算每个用户的物品共现情况
        for idx1, (item1, time1) in enumerate(item_time_list):
            for idx2, (item2, time2) in enumerate(item_time_list):
                if item1 == item2:
                    continue
                
                key = (item1, item2) if item1 < item2 else (item2, item1)
                user_co_items[key].append(user)  # 记录用户组对应的物品

    # 2. 计算物品之间的 Swing 相似度
    item_sim: Dict[int, Dict[int, float]] = defaultdict()
    alpha = 5.0  # Swing 公式的平滑参数

    for (item1, item2), co_users in tqdm(user_co_items.items(), desc="Computing Swing similarity"):
        num_co_users = len(co_users)  # 共同点击该对物品的用户数，即|I_u ∩ I_v|
        overlap = 1.0 / (alpha + num_co_users)

        item_sim.setdefault(item1, {})
        item_sim.setdefault(item2, {})

        for user in co_users:            
            item_sim[item1][item2] = item_sim[item1].get(item2, 0.0) + overlap
            item_sim[item2][item1] = item_sim[item2].get(item1, 0.0) + overlap

    # 3. 归一化相似性矩阵
    for item1, related_items in tqdm(item_sim.items(), desc="Normalizing similarity matrix"):
        for item2, sim_value in related_items.items():
            item_sim[item1][item2] = sim_value / math.sqrt(item_user_cnt[item1] * item_user_cnt[item2])

    # 4. 构建倒排索引
    inverted_index: Dict[int, List[Tuple[int, float]]] = defaultdict(list)
    for item, related_items in tqdm(item_sim.items(), desc="Building user inverted index"):
        topk_items = heapq.nlargest(len(related_items), related_items.items(), key=lambda x: x[1])
        inverted_index[item] = [(j, sim) for j, sim in topk_items]

    # 5. 存储计算结果
    mode = "offline" if offline else "online"
    with open(f"{save_path}/swing_i2i_sim_{mode}.pkl", "wb") as f:
        pickle.dump(item_sim, f)
    
    with open(f"{save_path}/swing_inverted_index_{mode}.pkl", "wb") as f:
        pickle.dump(inverted_index, f)

    return inverted_index

In [10]:
# 调用Swing召回通道进行新闻召回
user_item_time_dict = get_user_item_time(train_df) # 用户点击行为时间线字典

In [13]:
inverted_index = swing_sim(user_item_time_dict, save_path, offline) # 构建物品相似物品的倒排索引

Building item-user interaction matrix: 100%|██████████| 869946/869946 [48:57<00:00, 296.19it/s]   


TypeError: first argument must be callable or None

#### 折叠起来

In [ ]:
def hot_item_recall(user_id : int,
                    user_item_time_dict: Dict[int, List[Tuple[int, int]]],
                    item_topk_click: List[int],
                    recall_item_num: int,
                    filter_hist: bool = True) -> Dict[int, List[int]]:
    """
    热门物品独立召回通道
    
    参数:
        :user_id: 用户ID
        user_item_time_dict: 用户历史行为字典 {user_id: [(item1, time1), ...]}
        item_topk_click: 全局热门物品列表（需按热度降序排列）
        recall_item_num: 每个用户召回数量
        filter_hist: 是否过滤用户历史已交互物品
        
    返回:
        Dict[int]: 用户召回结果字典 {item1, item2, ...}
    """
    recall_dict = set()

    # 获取用户历史交互物品用于去重
    hist_ids = {item_id for item_id, _ in user_item_time_dict[user_id]} if filter_hist else set()
        
    # 生成召回列表（按全局热度排序）
    for item_id in item_topk_click:
        if item_id not in hist_ids:
            recall_dict.add(item_id)
            if len(recall_dict) == recall_item_num:
                    break

    return recall_dict

In [ ]:
def itemcf_recall(user_id, user_item_time_dict, itemcf_inverted_index, sim_item_topk, recall_item_num):
    """
    基于物品协同过滤的召回通道。
    :param user_id: 用户ID
    :param user_item_time_dict: 用户历史点击记录 {user_id: [(item_id, timestamp), ...]}
    :param itemcf_inverted_index: 物品倒排索引表 {item_id: [(similar_item_id, similarity_score), ...]}
    :param sim_item_topk: 选择每个物品最相似的前K个物品
    :param recall_item_num: 召回的物品数量
    :return: 召回的物品列表 {item1, item2, ...}
    """
    user_hist_items = {click_article_id for click_article_id, click_timestamp in user_item_time_dict.get(user_id, [])}
    item_rank = defaultdict(float)
    
    # 遍历用户点击过的物品
    for item in user_hist_items:
        # 获取与当前物品最相似的topK物品
        for similar_item, similarity in itemcf_inverted_index[item][:sim_item_topk]:
            # 过滤掉用户已经点击过的物品
            if similar_item in user_hist_items:
                continue
            
            # 计算用户的兴趣得分
            item_rank[similar_item] += similarity

    return {item for item, like_socre in sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]}

In [ ]:
def merge_multi_recall(user_item_time_dict, itemcf_inverted_index, item_topk_click, all_click_df, recall_methods):
    """
    多路召回，支持不同召回方法的融合。
    
    :param user_item_time_dict: dict, 用户行为记录 {user_id: [(item_id, timestamp), ...]}
    :param itemcf_inverted_index: dict, 物品倒排索引 {item_id: [(sim_item_id, similarity), ...]}
    :param item_topk_click: list, 全局热门物品（按热度降序排列）
    :param all_click_df: DataFrame, 用户点击数据
    :param recall_methods: dict, 召回方法及对应的召回数量 {召回方法名: 召回数量}
    :return: dict, 用户召回结果 {user_id: [召回物品列表]}
    """

    # 存储每个用户的召回列表（使用 set 去重）
    user_recall_items_dict = defaultdict(set)

    # 遍历所有用户
    for user_id in tqdm(all_click_df['user_id'].unique(), desc="Multi-Channel Recall"):

        # 遍历不同召回方法
        for method, recall_num in recall_methods.items():
            if method == "itemcf":
                recall_items = itemcf_recall(
                    user_id=user_id,
                    user_item_time_dict=user_item_time_dict,
                    itemcf_inverted_index=itemcf_inverted_index,
                    sim_item_topk=10,  # 选取最相似的10个物品
                    recall_item_num=recall_num  # 召回指定数量的物品
                )
            elif method == "hot":
                recall_items = hot_item_recall(
                    user_id=user_id,
                    user_item_time_dict=user_item_time_dict,
                    item_topk_click=item_topk_click,
                    recall_item_num=recall_num
                )
            else:
                print(f"⚠️ 未知召回方法: {method}，请检查代码！")
                continue
            
            # 合并召回结果（去重）
            user_recall_items_dict[user_id].update(recall_items)

    return user_recall_items_dict